In [1]:
# Importing Libraries

import pandas as pd
import numpy as np
import os

# Importing datasets

In [2]:
df_prods = pd.read_csv(r'/Users/alejandroguillen/Documents/Instacart Basket Analysis/02 Data/Original Data/products.csv', index_col = False)

In [3]:
df_ords = pd.read_csv (r'/Users/alejandroguillen/Documents/Instacart Basket Analysis/02 Data/Prepared Data/orders_wrangled.csv', index_col = False)

# Consistency checks for df_prods

In [4]:
# Exploring data: first 5 rows

df_prods.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [5]:
# Last 5 rows
df_prods.tail()

,product_id,product_name,aisle_id,department_id,prices
49688,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5,5.3
49689,49685,En Croute Roast Hazelnut Cranberry,42,1,3.1
49690,49686,Artisan Baguette,112,3,7.8
49691,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8,4.7
49692,49688,Fresh Foaming Cleanser,73,11,13.5


In [6]:
# Column names

df_prods.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices'], dtype='object')

In [7]:
# Data types

df_prods.dtypes

product_id         int64
product_name      object
aisle_id           int64
department_id      int64
prices           float64
dtype: object

# Checking mixed_type data

In [8]:
for col in df_prods.columns.tolist(): 
    weird = (df_prods[[col]].applymap(type) != df_prods[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_prods[weird]) > 0:
        print (col)

product_name


### "product_name" contains mixed_type data.

## Checking missing values

In [9]:
# Find missing values
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

### 16 missing values in product_name

In [10]:
# Create subset containing missing values
df_nan = df_prods[df_prods['product_name'].isnull() == True]

In [11]:
# View missing values subset
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


### Records with missing product names will be filtered out and a new dataframe created.

In [12]:
# Find row and column numbers for df_prods
df_prods.shape

(49693, 5)

In [13]:
# Create dataframe without missing values in 'product_name' column
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [14]:
# Compare row and column numbers for new dataframe
df_prods_clean.shape

(49677, 5)

In [15]:
# Check again for mixed-type data after filtering out missing values
for col in df_prods_clean.columns.tolist():
    weird = (df_prods_clean[[col]].applymap(type) != df_prods_clean[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_prods_clean[weird]) > 0:
        print (col)

# Checking duplicates

In [16]:
# Find full duplicates for df_prods_clean
df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [17]:
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


### 5 full duplicates found which will be dropped

In [18]:
# Create dataframe with duplicates removed
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [19]:
# Compare row and column numbers for dataframe with duplicates removed
df_prods_clean_no_dups.shape

(49672, 5)

# Descriptive statistics

In [20]:
# Descriptive Statistics for df_prods_clean_no_dups
df_prods_clean_no_dups.describe()

,product_id,aisle_id,department_id,prices
count,49672.000000,49672.000000,49672.000000,49672.000000
mean,24850.349775,67.762442,11.728942,9.993282
std,14340.705287,38.315784,5.850779,453.615536
min,1.000000,1.000000,1.000000,1.000000
25%,12432.750000,35.000000,7.000000,4.100000
50%,24850.500000,69.000000,13.000000,7.100000
75%,37268.250000,100.000000,17.000000,11.100000
max,49688.000000,134.000000,21.000000,99999.000000


### Max value for prices seems high.

In [21]:
# Record with max 'prices' value
df_prods_clean_no_dups.loc[df_prods_clean_no_dups['prices'] == 99999]

,product_id,product_name,aisle_id,department_id,prices
33666,33664,2 % Reduced Fat Milk,84,16,99999.0


### Price for milk is evidently overpriced. Potential error.

In [22]:
# Check for products that cost over $100
df_prods_clean_no_dups[df_prods_clean_no_dups["prices"]> 100.0]

,product_id,product_name,aisle_id,department_id,prices
21554,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0
33666,33664,2 % Reduced Fat Milk,84,16,99999.0


### These prices are incorrect. Will be replaced.

In [23]:
# Replace 'prices' value for 'product_id' 21553 with 1.49000 and 'product_id' 33664 with 9.99  
df_prods_clean_no_dups = df_prods_clean_no_dups.replace ({"prices":{99999.0: 9.99, 14900.0:1.49 }})

In [24]:
# Checking changes
df_prods_clean_no_dups[df_prods_clean_no_dups["product_id"]==21553]

,product_id,product_name,aisle_id,department_id,prices
21554,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,1.49


In [25]:
df_prods_clean_no_dups[df_prods_clean_no_dups["product_id"]==33664]

,product_id,product_name,aisle_id,department_id,prices
33666,33664,2 % Reduced Fat Milk,84,16,9.99


### Changes have been succesfully made. 

In [26]:
# Export df_prods_clean_no_dups
df_prods_clean_no_dups.to_csv(r'/Users/alejandroguillen/Documents/Instacart Basket Analysis/02 Data/Prepared Data/two_products_checked.csv')

# Consistency checks for df_ords

In [27]:
# First 5 rows
df_ords.head()

,Unnamed: 0,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
0,0,2539329,1,prior,1,2,8,NaN
1,1,2398795,1,prior,2,3,7,15.0
2,2,473747,1,prior,3,3,12,21.0
3,3,2254736,1,prior,4,4,7,29.0
4,4,431534,1,prior,5,4,15,28.0


In [28]:
df_ords = df_ords.drop(columns = ['Unnamed: 0'])

In [29]:
df_ords.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [30]:
# Last 5 rows

df_ords.tail()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0
3421082,272231,206209,train,14,6,14,30.0


In [31]:
# Column names 
df_ords.columns

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_last_order'],
      dtype='object')

In [32]:
# Number of rows and columns
df_ords.shape

(3421083, 7)

In [33]:
# Data types
df_ords.dtypes

order_id                   int64
user_id                    int64
eval_set                  object
order_number               int64
orders_day_of_week         int64
order_hour_of_day          int64
days_since_last_order    float64
dtype: object

# Checking mixed-type data

In [34]:
for col in df_ords.columns.tolist():
    weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_ords[weird]) > 0:
        print (col)
    

### No mixed-type data found

# Cheking missing values

In [35]:
df_ords.isnull().sum()

order_id                      0
user_id                       0
eval_set                      0
order_number                  0
orders_day_of_week            0
order_hour_of_day             0
days_since_last_order    206209
dtype: int64

In [36]:
# Create subset to show missing values
df_ords_nan = df_ords[df_ords['days_since_last_order'].isnull() == True]

In [37]:
df_ords_nan


,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
0,2539329,1,prior,1,2,8,NaN
11,2168274,2,prior,1,2,11,NaN
26,1374495,3,prior,1,1,14,NaN
39,3343014,4,prior,1,6,11,NaN
45,2717275,5,prior,1,3,12,NaN
...,...,...,...,...,...,...,...
3420930,969311,206205,prior,1,4,12,NaN
3420934,3189322,206206,prior,1,3,18,NaN
3421002,2166133,206207,prior,1,6,19,NaN
3421019,2227043,206208,prior,1,1,15,NaN


### The "days_since_last_order" missing values match with the order numbers "1". Which means that is the customer's first order and therefore there is no data available before that.

In [45]:
# Create flag column
condition = df_ords_nan['days_since_last_order'].isnull() & (df_ords_nan['order_number'] == 1)
df_ords_nan.loc[condition, 'customer_type'] = 'New Customer'
df_ords_nan.loc[~condition, 'customer_type'] = 'Returning Customer'

In [46]:
df_ords_nan.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,customer_type
0,2539329,1,prior,1,2,8,NaN,New Customer
11,2168274,2,prior,1,2,11,NaN,New Customer
26,1374495,3,prior,1,1,14,NaN,New Customer
39,3343014,4,prior,1,6,11,NaN,New Customer
45,2717275,5,prior,1,3,12,NaN,New Customer


In [47]:
df_ords_nan.tail()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,customer_type
3420930,969311,206205,prior,1,4,12,NaN,New Customer
3420934,3189322,206206,prior,1,3,18,NaN,New Customer
3421002,2166133,206207,prior,1,6,19,NaN,New Customer
3421019,2227043,206208,prior,1,1,15,NaN,New Customer
3421069,3154581,206209,prior,1,3,11,NaN,New Customer


In [48]:
# Show rows with order numbers 1
order_number_1 = df_ords_nan.loc[df_ords_nan['customer_type'] == 'New Customer']
print("Rows with order number 1:")
print(order_number_1)

Rows with order number 1:
         order_id  user_id eval_set  order_number  orders_day_of_week  \
0         2539329        1    prior             1                   2   
11        2168274        2    prior             1                   2   
26        1374495        3    prior             1                   1   
39        3343014        4    prior             1                   6   
45        2717275        5    prior             1                   3   
...           ...      ...      ...           ...                 ...   
3420930    969311   206205    prior             1                   4   
3420934   3189322   206206    prior             1                   3   
3421002   2166133   206207    prior             1                   6   
3421019   2227043   206208    prior             1                   1   
3421069   3154581   206209    prior             1                   3   

         order_hour_of_day  days_since_last_order customer_type  
0                        8     

In [49]:
order_number_1.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,customer_type
0,2539329,1,prior,1,2,8,NaN,New Customer
11,2168274,2,prior,1,2,11,NaN,New Customer
26,1374495,3,prior,1,1,14,NaN,New Customer
39,3343014,4,prior,1,6,11,NaN,New Customer
45,2717275,5,prior,1,3,12,NaN,New Customer


In [50]:
# Show rows with order numbers higher than 1
order_numbers_higher = df_ords_nan.loc[df_ords_nan['customer_type'] == 'Returning Customer']


In [52]:
order_numbers_higher.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,customer_type


In [39]:
# Address missing values

### I have decided not to impute or remove this data since statistical calculations can still be made. In addition, is good to know for further context in the analysis that these orders were the first orders made by these customers.

# Checking duplicates

In [40]:
df_ords_dups = df_ords[df_ords.duplicated()]

In [41]:
df_ords_dups

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order


### No duplicates found.

# Descriptive statistics

In [42]:
df_ords.describe()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


### Statistics seem normal.

In [ ]:
# Export df_ords 
df_ords.to_csv(r'/Users/alejandroguillen/Documents/Instacart Basket Analysis/02 Data/Prepared Data/two_orders_checked.csv', index = False)